Requirements:
- create a .env file with your sap credentials
- install crewai

In [2]:
! pip install crewai

  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached chromadb-1.1.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (7.2 kB)
  Using cached instructor-1.12.0-py3-none-any.whl.metadata (11 kB)
  Using cached json_repair-0.25.2-py3-none-any.whl.metadata (7.9 kB)
  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.38.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached pdfplumber-0.11.7-py3-none-any.whl.metadata (42 kB)
  Using cached portalocker-2.7.0-py2.py3-none-any.whl.metadata (6.8 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached tomli_w-1.2.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached tomli-2.3.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (10 kB)
  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached pybase64-1.4.2-cp312-cp312-macosx_11_0_arm64.w

For more information about CrewAi framework do to [docs](https://docs.crewai.com/en/introduction)

In [ ]:
from crewai import Agent, Task, Crew
from crewai.tools import tool
from dotenv import load_dotenv

Load your environment. SAP credentials will be automatically retrieved to litellm from the environment variables

In [ ]:
load_dotenv()

Define the tool function

In [ ]:
@tool("get_weather")
def get_weather(city: str) -> str:
    """Moke function"""
    city_normalized = city.lower().replace(" ", "")

    mock_weather_db = {
        "newyork": "The weather in New York is sunny with a temperature of 25°C.",
        "london": "It's cloudy in London with a temperature of 15°C.",
        "tokyo": "Tokyo is experiencing light rain and a temperature of 18°C.",
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return f"The weather in {city} is sunny with a temperature of 20°C."

User input

In [ ]:
city = input("Input city: ")

Define the agent. Put the SAP model in model parameter. The CrewAI use the LiteLLM lib inside

In [ ]:
agent = Agent(
    role="Weather presenter",
    goal=f"Prepare a couple sentences TV speach about weather in the {city}, "
         f"using information from run the get_weather tool",
    backstory="You are the weather presenter on TV",
    llm="sap/gpt-4o",
    tools=[get_weather],
    allow_delegation=False,
)

Define the agent task

In [ ]:
agent_task = Task(
    description=(
        f"Write a couple sentences for TV weather report in {city}, that will be include small jok"
    ),
    expected_output=(
        "Good quality text of two sentences about weather and with small jok"
    ),
    agent=agent,
)

Create the crew with the agent and the task

In [ ]:
crew = Crew(
    agents=[agent],
    tasks=[agent_task],
    verbose=True,
)

Run the crew

In [ ]:
result = crew.kickoff()
print("\n📘 Result:\n", result)